### History

Tries to do CNNs from scratch. Works, but relies on PyTorch operations. Will eventually rewrite in pure PyTorch.

## Convolution Demos

**Note:** Old losses and layers code.

In [4]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Add Lincoln to system path
import sys
sys.path.append("/Users/seth/development/lincoln/")

In [5]:
import torch
from torch import Tensor

import lincoln
from lincoln.layers import Dense
from lincoln.losses import LogSoftmaxLoss, LogSigmoidLoss, MeanSquaredError
from lincoln.optimizers import SGD
from lincoln.operations.activations import Sigmoid, LogSigmoid
from lincoln.network import NeuralNetwork
from lincoln.train import Trainer

In [6]:
from lincoln.layers import Conv2D
from lincoln.operations.activations import ReLU

In [8]:
from torchvision.datasets import MNIST
mnist_trainset = MNIST(root="../exploratory/data/", train=True, download=False, transform=None)
mnist_testset = MNIST(root="../exploratory/data/", train=False, download=False, transform=None)

#### One hot encode labels

In [9]:
data = mnist_trainset
num_labels = len(data.train_labels)
train_labels = torch.zeros(num_labels, 10)
for i in range(num_labels):
    train_labels[i][data.train_labels[i]] = 1
train_labels.shape

torch.Size([60000, 10])

In [10]:
data = mnist_testset
num_labels = len(data.test_labels)
test_labels = torch.zeros(num_labels, 10)
for i in range(num_labels):
    test_labels[i][data.test_labels[i]] = 1
test_labels.shape

torch.Size([10000, 10])

In [11]:
mnist_train = mnist_trainset.train_data.type(torch.float32).unsqueeze(3) / 255.0
mnist_test = mnist_testset.test_data.type(torch.float32).unsqueeze(3) / 255.0

In [12]:
X_train = mnist_train
X_test = mnist_test
y_train = train_labels
y_test = test_labels

In [13]:
def _conv_transform(input_: Tensor) -> Tensor:

    return input_.view(input_.shape[0],
                       input_.shape[1] * input_.shape[2] * input_.shape[3])

X_train_flat = _conv_transform(X_train)
X_test_flat = _conv_transform(X_test)

In [14]:
conv_nn_ll = NeuralNetwork(
    layers=[Dense(neurons=250, 
                  activation=Sigmoid()),
            Dense(neurons=50, 
                  activation=Sigmoid()),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss())

In [15]:
optimizer = SGD(0.1)

In [16]:
trainer = Trainer(conv_nn_ll, optimizer)

In [17]:
import time
start = time.time()
trainer.fit(X_train_flat, y_train, X_test_flat, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=100,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")

Validation loss after 1 epochs is 1434814.625
Training took 11.1 seconds.


In [18]:
def eval_nn(nn, X_test, y_test):
    test_preds = nn.forward(X_test)
    
    test_preds_l = torch.argmax(test_preds, dim=1)
    
    test_vals_l = torch.argmax(y_test, dim=1)
    
    return torch.sum(test_preds_l == test_vals_l)

In [19]:
eval_nn(conv_nn_ll, X_test_flat, y_test)

tensor(9277)

92.77% accuracy

## Need to Redo rest of this

In [ ]:
X_train_perm = X_train.permute(0, 3, 1, 2)
X_test_perm = X_test.permute(0, 3, 1, 2)

In [ ]:
# settings
pytorch = True

In [ ]:
conv_nn_logloss = NeuralNetwork(
    layers=[Conv2D(param_size=3,
                   out_channels=24,
                   activation=ReLU(),
                   pytorch=pytorch),
            Conv2D(param_size=3,
                   out_channels=30,
                   activation=ReLU(),
                   pytorch=pytorch,
                   flatten=True),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(),
            optimizer = SGD(0.1))

In [ ]:
import time
start = time.time()
conv_nn_logloss.fit(X_train_perm, y_train, X_test_perm, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=100,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")

In [ ]:
eval_nn(conv_nn_logloss, X_test_perm, y_test)

### Testing network without PyTorch layers 

In [ ]:
# %load_ext autoreload
# %autoreload 2

#### No Cython

In [ ]:
pytorch = False
cython = False

In [ ]:
conv_nn_normal = NeuralNetwork(
    layers=[Conv2D(param_size=3,
                   out_channels=24,
                   activation=ReLU(),
                   pytorch=pytorch,
                   cython=cython),
            Conv2D(param_size=3,
                   out_channels=30,
                   activation=ReLU(),
                   pytorch=pytorch,
                   cython=cython,
                   flatten=True),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(),
            optimizer = SGD(0.1))

In [ ]:
import time
start = time.time()
conv_nn_normal.fit(X_train_perm, y_train, X_test_perm, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=1,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")

#### Cython

In [ ]:
pytorch = False
cython = True

In [ ]:
conv_nn_cython = NeuralNetwork(
    layers=[Conv2D(param_size=3,
                   out_channels=24,
                   activation=ReLU(),
                   pytorch=pytorch,
                   cython=cython),
            Conv2D(param_size=3,
                   out_channels=30,
                   activation=ReLU(),
                   pytorch=pytorch,
                   cython=cython,
                   flatten=True),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(),
            optimizer = SGD(0.1))

In [ ]:
import time
start = time.time()
conv_nn_cython.fit(X_train_perm, y_train, X_test_perm, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=1,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")